<a href="https://colab.research.google.com/github/Jurgo001/TH_TriTueNhanTao/blob/main/Buoi04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# -*- coding: utf-8 -*-
# Cài đặt thư viện ColabTurtle (Chỉ cần chạy lệnh này một lần trong Colab)
# !pip install ColabTurtle

import numpy as np      # Xử lý tính toán mảng (Ví dụ: tính bậc, sắp xếp)
import math             # Cung cấp các hàm toán học (Ví dụ: tính sin/cos/radian cho vẽ đồ thị)
import itertools        # Tạo ra các hoán vị (permutations) cho thuật toán TSP
from ColabTurtle.Turtle import * # Thư viện dùng để trực quan hóa đồ thị

# --- I. DỮ LIỆU ĐỒ THỊ ---

TEN_DINH = "ABCDEF"     # Tên các đỉnh
SO_DINH = len(TEN_DINH) # Số lượng đỉnh (6)
INF = float('inf')      # Đại diện cho vô cực, dùng trong tính toán khoảng cách TSP

# 1. Ma trận kề (Dùng cho TÔ MÀU)
MA_TRAN_KE = [
    [0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0], [1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 1, 1], [0, 0, 1, 1, 0, 1], [0, 0, 0, 1, 1, 0]
]

# 2. Ma trận trọng số (Dùng cho TSP)
MA_TRAN_TRONG_SO = [
    [0, 12, 10, 0, 0, 0], [12, 0, 8, 15, 0, 0], [10, 8, 0, 18, 20, 0],
    [0, 15, 18, 0, 10, 25], [0, 0, 20, 10, 0, 30], [0, 0, 0, 25, 30, 0]
]

print("Đã load dữ liệu cho 6 đỉnh:", TEN_DINH)
print("-" * 30)

# --- II. TÔ MÀU ĐỒ THỊ (THUẬT TOÁN GREEDY) ---

def to_mau_do_thi(MA_TRAN):
    """Thực hiện tô màu đồ thị bằng thuật toán tham lam (sắp xếp theo bậc giảm dần)."""

    # 1. Tính toán bậc và sắp xếp đỉnh theo bậc giảm dần
    bac_dinh = [sum(MA_TRAN[i]) for i in range(SO_DINH)] # Tính bậc (tổng hàng) của từng đỉnh
    chi_so_sap_xep = np.argsort(bac_dinh)[::-1]          # Lấy chỉ số các đỉnh theo bậc giảm dần
    dinh_sap_xep = [TEN_DINH[i] for i in chi_so_sap_xep] # Danh sách tên đỉnh đã sắp xếp (VD: C, D, B, E, A, F)

    DS_mau_kha_dung = {dinh: ["Blue", "Red", "Yellow", "Green"] for dinh in TEN_DINH} # Cấp 4 màu cho mỗi đỉnh
    ket_qua_to_mau = {} # Dictionary lưu kết quả cuối cùng {Đỉnh: Màu}

    # 2. Tô màu
    for ten_dinh in dinh_sap_xep:
        # Nếu đỉnh còn màu khả dụng
        if DS_mau_kha_dung[ten_dinh]:
            mau_duoc_chon = DS_mau_kha_dung[ten_dinh][0]     # Chọn màu đầu tiên trong danh sách
            ket_qua_to_mau[ten_dinh] = mau_duoc_chon          # Gán màu đã chọn cho đỉnh

            # Loại bỏ màu này khỏi các đỉnh kề
            chi_so_dinh_hien_tai = TEN_DINH.index(ten_dinh)  # Lấy chỉ số của đỉnh hiện tại (VD: C là 2)

            for j in range(SO_DINH):
                if MA_TRAN[chi_so_dinh_hien_tai][j] == 1:    # Kiểm tra đỉnh j có kề với đỉnh hiện tại không
                    ten_dinh_ke = TEN_DINH[j]
                    if mau_duoc_chon in DS_mau_kha_dung[ten_dinh_ke]:
                        DS_mau_kha_dung[ten_dinh_ke].remove(mau_duoc_chon) # Xóa màu khỏi đỉnh kề
        else:
            ket_qua_to_mau[ten_dinh] = "Không đủ màu"

    return ket_qua_to_mau

# Thực thi tô màu và in kết quả
KetQuaMau = to_mau_do_thi(MA_TRAN_KE)

print("KẾT QUẢ TÔ MÀU ĐỒ THỊ:")
for t, w in sorted(KetQuaMau.items()):
    print(f"Đỉnh {t} = {w}")

# --- III. VẼ ĐỒ THỊ VÀ TÔ MÀU (COLABTURTLE) ---

def ve_do_thi_va_to_mau(MA_TRAN, KetQuaMau):
    """Vẽ đồ thị và tô màu đỉnh."""
    initializeTurtle()
    speed(5); bgcolor("white"); pensize(2) # Thiết lập màn hình vẽ: tốc độ, màu nền, độ dày bút

    trung_tam_x, trung_tam_y, ban_kinh = 400, 250, 150 # Thiết lập tâm và bán kính vòng tròn
    vi_tri_dinh = {}
    goc_buoc = 360 / SO_DINH # Góc lệch giữa các đỉnh trên vòng tròn

    # Tính tọa độ
    for i in range(SO_DINH):
        goc_radian = math.radians(i * goc_buoc)
        x = trung_tam_x + math.cos(goc_radian) * ban_kinh
        y = trung_tam_y + math.sin(goc_radian) * ban_kinh
        vi_tri_dinh[TEN_DINH[i]] = (x, y) # Lưu tọa độ (x, y) của đỉnh

    # Vẽ cạnh
    color("gray")
    for i in range(SO_DINH):
        for j in range(i + 1, SO_DINH):
            if MA_TRAN[i][j] == 1: # Nếu có cạnh
                toa_do_dau = vi_tri_dinh[TEN_DINH[i]]
                toa_do_cuoi = vi_tri_dinh[TEN_DINH[j]]
                penup(); goto(toa_do_dau); pendown(); goto(toa_do_cuoi) # Nhấc bút, đi đến đầu, đặt bút, vẽ đến cuối

    # Vẽ đỉnh và tô màu
    for ten_dinh in TEN_DINH:
        x, y = vi_tri_dinh[ten_dinh]
        mau_to = KetQuaMau.get(ten_dinh, "white").lower() # Lấy màu đã tô (dùng lower() vì ColabTurtle chỉ nhận màu chữ thường)

        penup(); goto(x, y); color(mau_to)

        # Vẽ chấm tròn (Đỉnh)
        old_size = pensize()
        pensize(30); pendown(); forward(0); pensize(old_size); penup() # forward(0) vẽ một chấm tròn

        # Viết tên đỉnh
        goto(x, y + 8); color("black")
        write(ten_dinh, align="center", font=(20, "Arial", "bold"))

    goto(50, 450) # Đưa rùa ra góc màn hình

print("-" * 30)
print("Bắt đầu vẽ đồ thị TÔ MÀU...")
ve_do_thi_va_to_mau(MA_TRAN_KE, KetQuaMau)


# --- IV. BÀI TOÁN NGƯỜI DU LỊCH (TSP - VÉT CẠN) ---

def tinh_tong_quang_duong(lo_trinh, TRONG_SO):
    """Tính tổng khoảng cách của lộ trình."""
    tong_kc = 0
    for i in range(len(lo_trinh) - 1):
        u = lo_trinh[i]; v = lo_trinh[i+1] # Lấy chỉ số đỉnh đi (u) và đỉnh đến (v)
        khoang_cach = TRONG_SO[u][v]
        if khoang_cach == 0: return INF     # Nếu không có đường đi (khoảng cách = 0), lộ trình này không hợp lệ
        tong_kc += khoang_cach              # Cộng dồn khoảng cách
    return tong_kc

# 1. Tính toán lộ trình tối ưu (Bắt đầu và kết thúc tại A - chỉ số 0)
dinh_con_lai = list(range(1, SO_DINH)) # [1, 2, 3, 4, 5] Tức là các đỉnh B, C, D, E, F
tat_ca_lo_trinh = itertools.permutations(dinh_con_lai) # Sinh ra tất cả hoán vị của các đỉnh còn lại

duong_di_ngan_nhat = []
khoang_cach_ngan_nhat = INF
dem_so_truong_hop = 0

for phuong_an in tat_ca_lo_trinh:
    dem_so_truong_hop += 1
    lo_trinh_hien_tai = [0] + list(phuong_an) + [0] # Tạo lộ trình khép kín: [A] -> [Hoán vị] -> [A]
    do_dai = tinh_tong_quang_duong(lo_trinh_hien_tai, MA_TRAN_TRONG_SO)

    if do_dai < khoang_cach_ngan_nhat: # Kiểm tra xem lộ trình hiện tại có ngắn hơn lộ trình tốt nhất không
        khoang_cach_ngan_nhat = do_dai
        duong_di_ngan_nhat = lo_trinh_hien_tai # Lưu lại lộ trình mới

# 2. In kết quả tính toán
ten_duong_di = [TEN_DINH[i] for i in duong_di_ngan_nhat]

print("\n--- BÀI TOÁN NGƯỜI DU LỊCH (TSP) ---")
print(f"Đã kiểm tra tổng cộng: {dem_so_truong_hop} trường hợp.")
print(f"-> LỘ TRÌNH TỐI ƯU: {' -> '.join(ten_duong_di)}")
print(f"-> TỔNG QUÃNG ĐƯỜNG: {khoang_cach_ngan_nhat} km")

# 3. Mô phỏng di chuyển

if khoang_cach_ngan_nhat != INF:
    initializeTurtle()
    speed(13); bgcolor("white"); pensize(2)

    trung_tam_x, trung_tam_y, ban_kinh = 400, 250, 150
    goc_buoc = 360 / SO_DINH

    # Khởi tạo tọa độ
    vi_tri_dinh_tsp = {i: (trung_tam_x + math.cos(math.radians(i * goc_buoc)) * ban_kinh,
                           trung_tam_y + math.sin(math.radians(i * goc_buoc)) * ban_kinh)
                        for i in range(SO_DINH)}

    # Dựng bản đồ có trọng số
    color("lightgray")
    for i in range(SO_DINH):
        for j in range(i + 1, SO_DINH):
            kc = MA_TRAN_TRONG_SO[i][j]
            if kc > 0:
                start = vi_tri_dinh_tsp[i]; end = vi_tri_dinh_tsp[j]

                penup(); goto(start); pendown(); goto(end)

                # Viết trọng số (khoảng cách)
                mid_x = (start[0] + end[0]) / 2; mid_y = (start[1] + end[1]) / 2
                penup(); goto(mid_x, mid_y); color("blue")
                write(str(kc), align="center", font=(10, "Arial", "normal")); color("lightgray")

        # Vẽ tên đỉnh (đã sửa lỗi NameError)
        x, y = vi_tri_dinh_tsp[i]; penup(); goto(x, y + 20); color("black")
        write(TEN_DINH[i], align="center", font=(15, "Arial", "bold"))

    # Mô phỏng đường đi tối ưu
    print("Bắt đầu mô phỏng giao hàng theo lộ trình tối ưu...")
    speed(3); color("red"); pensize(3) # Thiết lập màu đỏ đậm cho đường đi tối ưu

    start_point = vi_tri_dinh_tsp[duong_di_ngan_nhat[0]]
    penup(); goto(start_point); pendown()

    for i in range(1, len(duong_di_ngan_nhat)):
        next_pos = vi_tri_dinh_tsp[duong_di_ngan_nhat[i]]
        goto(next_pos) # Rùa di chuyển theo lộ trình tối ưu

    print("Giao hàng hoàn tất!")

penup(); goto(50, 450)

Đã load dữ liệu cho 6 đỉnh: ABCDEF
------------------------------
KẾT QUẢ TÔ MÀU ĐỒ THỊ:
Đỉnh A = Red
Đỉnh B = Yellow
Đỉnh C = Blue
Đỉnh D = Red
Đỉnh E = Yellow
Đỉnh F = Blue
------------------------------
Bắt đầu vẽ đồ thị TÔ MÀU...



--- BÀI TOÁN NGƯỜI DU LỊCH (TSP) ---
Đã kiểm tra tổng cộng: 120 trường hợp.
-> LỘ TRÌNH TỐI ƯU: A -> B -> D -> F -> E -> C -> A
-> TỔNG QUÃNG ĐƯỜNG: 112 km


Bắt đầu mô phỏng giao hàng theo lộ trình tối ưu...
Giao hàng hoàn tất!
